## Bitcoin strategy Example

In [1]:
import os

os.chdir("..")

In [2]:
from datetime import date
from pxtrade import Trade, Strategy, History, Backtest
from pxtrade.assets import reset, Cash, FxRate, Portfolio
from pxtrade.events.yahoo import load_yahoo_prices
from pxtrade.compliance import Compliance, UnitLimit

In [3]:
# initialise assets
reset()
usd = Cash("USD")
btc = Cash("BTC")
btcusd = FxRate("BTCUSD")
portfolio = Portfolio("USD")
starting_value = int(1e6)
portfolio.transfer(usd, starting_value)

In [4]:
# impose limit (either short or long) for BTC as 100 coins
portfolio.compliance = Compliance().add_rule(
    UnitLimit(btc, 100)
)

In [5]:
class BTCStrategy(Strategy):
    def generate_trades(self):
        """ Continue to buy one coin until we reach our position limit. """
        return Trade(portfolio, btc, +1)


In [6]:
backtest = Backtest(BTCStrategy())
history = History(portfolio)

In [7]:
start_date = date(2020, 1, 1)
end_date = date(2020, 9, 1)
load_yahoo_prices(
    btcusd,
    backtest,
    start_date=start_date,
    end_date=end_date,
)

In [8]:
backtest.run()
df = history.get()

df

,BTC,BTCUSD,Portfolio,Portfolio_BTC,Portfolio_USD,USD
2020-01-01,1.0,7200.175781,1.000000e+06,1.0,992799.824219,1.0
2020-01-02,1.0,6985.469238,9.997853e+05,2.0,985814.354980,1.0
2020-01-03,1.0,7344.883301,1.000504e+06,3.0,978469.471680,1.0
2020-01-06,1.0,7769.216797,1.001777e+06,4.0,970700.254883,1.0
2020-01-07,1.0,8163.691895,1.003355e+06,5.0,962536.562988,1.0
...,...,...,...,...,...,...
2020-08-25,1.0,11487.557617,1.328559e+06,100.0,179802.971680,1.0
2020-08-26,1.0,11323.315430,1.312135e+06,100.0,179802.971680,1.0
2020-08-27,1.0,11541.054688,1.333908e+06,100.0,179802.971680,1.0
2020-08-30,1.0,11679.316406,1.347735e+06,100.0,179802.971680,1.0


In [9]:
import cufflinks as cf

columns = ["Portfolio", "Portfolio_USD", "BTCUSD"]

df[columns].iplot(
    secondary_y="BTCUSD",
    title="Portfolio Value",
)

***